In [ ]:
# --- IMPORTS & CONFIGURATION ---
import requests
import json
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# ⚠️ Set your API keys from environment
LINEAR_API_KEY = os.getenv("LINEAR_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# --- GLOBAL SETTINGS ---
LINEAR_API_URL = "https://api.linear.app/graphql"
LINEAR_HEADERS = {
    "Authorization": LINEAR_API_KEY, 
    "Content-Type": "application/json"
}

# Initialize OpenAI client
try:
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
except Exception as e:
    print(f"Warning: Could not initialize OpenAI client. AI features will fail. Error: {e}")
    openai_client = None


In [ ]:
# -----------------------------------------------------------------
#  SECTION 1: CORE API & HELPER FUNCTIONS
# -----------------------------------------------------------------

def run_linear_query(query: str, variables: dict = None):
    """
    (Helper) A generic function to run any query or mutation against the Linear API.
    """
    payload = {'query': query}
    if variables:
        payload['variables'] = variables
    
    try:
        response = requests.post(LINEAR_API_URL, headers=LINEAR_HEADERS, json=payload)
        response.raise_for_status()
        
        json_response = response.json()
        if "errors" in json_response:
            print(f"❌ GraphQL Error: {json_response['errors']}")
            return None
        return json_response
        
    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error: {e.status_code} - {e.response.text}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Request Error: {e}")
    return None

def get_team_id(team_key: str) -> str | None:
    """
    (Helper) Fetches the unique UUID of a Linear team (e.g., "LIN").
    """
    print(f"🔎 Fetching ID for team '{team_key}'...")
    query = """
        query GetTeamId($teamKey: String!) {
          team(id: $teamKey) {
            id
            name
          }
        }
    """
    variables = {"teamKey": team_key}
    data = run_linear_query(query, variables)
    
    if data and data.get('data', {}).get('team'):
        team_id = data['data']['team']['id']
        print(f"✅ Found Team: {data['data']['team']['name']} (ID: {team_id})")
        return team_id
    else:
        print(f"❌ Error: Could not find team with key '{team_key}'.")
        return None


In [ ]:
# -----------------------------------------------------------------
#  SECTION 2: MODULAR PROJECT FUNCTIONS
# -----------------------------------------------------------------

def create_project(team_id: str, project_name: str, project_summary: str):
    """
    1. CREATES THE PROJECT
    Sets the required TITLE (name) and SUMMARY (description).
    Returns the new project object, including its ID.
    """
    print(f"🚀 Creating base project: '{project_name}'...")
    mutation = """
        mutation ProjectCreate($name: String!, $description: String!, $teamIds: [String!]!) {
          projectCreate(input: {
            name: $name,
            description: $description,
            teamIds: $teamIds,
            state: "planned"
          }) {
            success
            project { 
              id, 
              name, 
              url 
            }
          }
        }
    """
    variables = {
        "name": project_name,
        "description": project_summary,
        "teamIds": [team_id]
    }
    data = run_linear_query(mutation, variables)
    
    if data and data.get('data', {}).get('projectCreate', {}).get('success'):
        project = data['data']['projectCreate']['project']
        print(f"✅ Base project created. (ID: {project['id']})")
        return project
    else:
        print("❌ Failed to create base project.")
        return None

def update_project_content(project_id: str, content: str):
    """
    2. SETS THE MAIN DESCRIPTION
    Updates the project with the rich-text 'content' (the main description).
    """
    print("✍️ Setting main project description (content)...")
    mutation = """
        mutation ProjectUpdate($projectId: String!, $input: ProjectUpdateInput!) {
          projectUpdate(id: $projectId, input: $input) {
            success
          }
        }
    """
    variables = {
        "projectId": project_id,
        "input": {"content": content}
    }
    data = run_linear_query(mutation, variables)
    if data and data.get('data', {}).get('projectUpdate', {}).get('success'):
        print("✅ Main description set.")
    else:
        print("❌ Failed to set main description.")

def add_project_update(project_id: str, body: str):
    """
    3. ADDS AN UPDATE
    Posts a new item to the 'Latest update' feed.
    """
    print("✍️ Posting to 'Latest update' feed...")
    mutation = """
        mutation ProjectUpdateCreate($projectId: String!, $body: String!) {
          projectUpdateCreate(input: {
            projectId: $projectId,
            body: $body
          }) {
            success
          }
        }
    """
    variables = {"projectId": project_id, "body": body}
    data = run_linear_query(mutation, variables)
    
    if data and data.get('data', {}).get('projectUpdateCreate', {}).get('success'):
        print(f"✅ Posted to update feed.")
    else:
        print(f"⚠️ Failed to post update.")

def create_project_milestone(project_id: str, name: str, description: str):
    """
    4. CREATES A MILESTONE
    Adds a new milestone and its description to the project.
    """
    print(f"🚀 Creating milestone: '{name}'...")
    mutation = """
        mutation MilestoneCreate($name: String!, $projectId: String!, $description: String) {
          projectMilestoneCreate(input: {
            name: $name,
            projectId: $projectId,
            description: $description
          }) {
            success
          }
        }
    """
    variables = {
        "projectId": project_id,
        "name": name,
        "description": description
    }
    data = run_linear_query(mutation, variables)
    
    if data and data.get('data', {}).get('projectMilestoneCreate', {}).get('success'):
        print("✅ Milestone created.")
    else:
        print("❌ Failed to create milestone.")

In [ ]:
# -----------------------------------------------------------------
#  SECTION 3: AI CONTENT GENERATOR (MODIFIED for N Milestones)
# -----------------------------------------------------------------

def ai_generate_project_components(prompt: str, num_milestones: int = 5) -> dict | None:
    """
    (AI) Generates all project components from a single prompt.
    It now generates a *list* of milestones, making it general.
    """
    if not openai_client: 
        print("❌ OpenAI client not initialized.")
        return None
        
    print(f"🤖 Asking OpenAI to generate a project with {num_milestones} milestones...")
    
    system_prompt = f"""
    You are an expert project manager. A user will provide a brief idea for a software project.
    Your job is to generate a complete starter pack for a Linear project.
    
    You MUST respond with a valid JSON object with exactly these 5 keys:
    1. "project_name": A concise, catchy, and professional project name.
    2. "project_summary": A **brief, one-sentence** project summary (for under the title). **MUST BE UNDER 255 CHARACTERS.**
    3. "project_content": A detailed main project description, written in Markdown.
    4. "project_update": A brief "kickoff" status post for the project's feed.
    5. "milestones": A JSON **array** of milestone objects. You must generate exactly {num_milestones} milestones. Each object in the array must have two keys: "name" and "description".
    """
    
    try:
        completion = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        details = json.loads(completion.choices[0].message.content)
        
        # Validate the new structure
        required_keys = ["project_name", "project_summary", "project_content", "project_update", "milestones"]
        
        if all(key in details for key in required_keys) and \
           isinstance(details["milestones"], list) and \
           len(details["milestones"]) == num_milestones: # Check if AI followed instructions
            
            print("🤖 AI generated all components successfully.")
            return details
        else:
            print(f"❌ AI response was invalid, missing keys, or wrong milestone count.")
            print(f"   Response: {details}")
            return None

    except Exception as e:
        print(f"❌ Error during OpenAI call or JSON parsing: {e}")
        return None

In [ ]:

# -----------------------------------------------------------------
#  SECTION 4: EXAMPLE USAGE
# -----------------------------------------------------------------

# --- EXAMPLE 1: MANUAL PROJECT CREATION (with 5 Milestones) ---

def run_manual_example():
    print("--- Running MANUAL Example ---")
    # --- 1. Define Your Content ---
    my_team_key = "LIN2" # ⚠️ Your Team Key (e.g., "LIN")

    my_title = "Manual Project (5 Milestones)"
    my_summary = "A test project with a full, 5-phase plan."
    my_update = "Kicking off this 5-milestone project!"
    my_main_content = """
# Project Specifications
This project is broken down into 5 distinct phases, each represented by a milestone.
"""

    # --- Define all 5 milestones in a list ---
    my_milestones = [
        { "name": "Phase 1: Research & Discovery", "desc": "Understand the problem and user needs." },
        { "name": "Phase 2: Design & Prototyping", "desc": "Create wireframes, mockups, and user flows." },
        { "name": "Phase 3: Development (Back-end)", "desc": "Build the API, database, and auth logic." },
        { "name": "Phase 4: Development (Front-end)", "desc": "Build the UI and connect to the API." },
        { "name": "Phase 5: Launch & QA", "desc": "Deploy to production and conduct final testing." }
    ]

    # --- 2. Get Team ID ---
    team_id = get_team_id(my_team_key)

    if team_id:
        # --- 3. Run the Functions in Order ---
        
        # Create the base project (Title + Summary)
        project = create_project(
            team_id=team_id,
            project_name=my_title,
            project_summary=my_summary
        )
        
        if project:
            project_id = project['id']
            
            # Add the Main Description
            update_project_content(
                project_id=project_id,
                content=my_main_content
            )
            
            # Add the first Update post
            add_project_update(
                project_id=project_id,
                body=my_update
            )
            
            # Loop through and create all 5 milestones
            print("\nCreating 5 milestones...")
            for milestone in my_milestones:
                create_project_milestone(
                    project_id=project_id,
                    name=milestone["name"],
                    description=milestone["desc"]
                )
            
            print("\n" + "="*30)
            print(f"🎉🎉🎉 Successfully created MANUAL project with 5 milestones!")
            print(f"View it here: {project['url']}")
            print("="*30)


In [ ]:

# --- EXAMPLE 2: AI-POWERED PROJECT CREATION (with 5 Milestones) ---

def run_ai_example():
    print("--- Running AI Example ---")
    # --- 1. Define Your Prompt ---
    my_team_key = "LIN2" # ⚠️ Your Team Key (e.g., "LIN")
    my_project_idea = "A dektop app that helps users track their daily water intake. It should have reminders and graphs."

    # --- 2. Get Team ID ---
    team_id = get_team_id(my_team_key)

    if team_id:
        # --- 3. Generate AI Content (ask for 5 milestones) ---
        details = ai_generate_project_components(
            prompt=my_project_idea,
            num_milestones=2 # <-- We specify 5 here!
        )
        
        if details:
            # --- 4. Run the Functions in Order ---
            
            # Create the base project (Title + Summary)
            project = create_project(
                team_id=team_id,
                project_name=details["project_name"],
                project_summary=details["project_summary"]
            )
            
            if project:
                project_id = project['id']
                
                # Add the Main Description
                update_project_content(
                    project_id=project_id,
                    content=details["project_content"]
                )
                
                # Add the first Update post
                add_project_update(
                    project_id=project_id,
                    body=details["project_update"]
                )
                
                # --- Loop through the list of milestones from the AI ---
                print("\nCreating AI-generated milestones...")
                for milestone in details["milestones"]:
                    create_project_milestone(
                        project_id=project_id,
                        name=milestone["name"],
                        description=milestone["description"]
                    )
                
                print("\n" + "="*30)
                print(f"🎉🎉🎉 Successfully created AI project with 5 milestones!")
                print(f"View it here: {project['url']}")
                print("="*30)


In [ ]:

# -----------------------------------------------------------------
#  SECTION 5: SCRIPT EXECUTION
# -----------------------------------------------------------------
# To run one of the examples, uncomment one of the lines below:

# run_manual_example()
run_ai_example() 
print("Script loaded. Call run_manual_example() or run_ai_example() to create a project.")

In [ ]:
pip install discord

In [ ]:
LINEAR_API_KEY


**ボットのロジック概要** :robot:

このボットは、`/checklinear` コマンドで手動実行された際、以下のチェックを行います。

> **1. アクティブなプロジェクトの検索**
> 以下の**両方**の条件を満たすプロジェクトを「アクティブ」と判断します：
> * **開始日 (Start Date)** が **今日以前**
> * **ターゲット日 (Target Date)** が **今日以降**
>
> **2. 未完了イシューの検索**
> 上記1のアクティブなプロジェクト内で、ステータスが **"Completed" (完了)** または **"Canceled" (キャンセル)** **以外** のイシューを検索します。
>
> **3. 通知メッセージの作成**
> 未完了イシューが1件以上見つかったアクティブなプロジェクトのみを一覧にし、各チームの期日とイシュー詳細を含むDiscordメッセージを生成して通知します。